## Preprocessing

In [2]:
label_group_map = {
     'F': 'F'
}

In [3]:
import numpy as np
def resample_interpolation(ts, fs_in, fs_out):
    if fs_out == fs_in:
        return ts
    else:
        x_old = np.linspace(0, 1, num=len(ts), endpoint=True)
        x_new = np.linspace(0, 1, num=fs_out, endpoint=True)
        return np.interp(x_new, x_old, ts)

In [ ]:
import os
def read_records(path):
     with open(os.path.join(path, 'RECORDS'), 'r') as fin:
         all_record_name = fin.read().split('\n')
     return all_record_name
read_records(r'D:\CodeProjects\VsCodePythonPro\MachinelearningClass\EX5\mit-bih-arrhythmia-database-1.0.0_data')

In [ ]:
import wfdb

sample_record = os.path.join(r'D:\CodeProjects\VsCodePythonPro\MachinelearningClass\EX5\mit-bih-arrhythmia-database-1.0.0_data', '100')
sample_annotation = wfdb.rdann(sample_record, 'atr').__dict__
sample_data = wfdb.rdsamp(sample_record)

#print(len(sample_annotation))
# print(sample_annotation['record_name'])
# print(sample_annotation['symbol'])
sample_annotation


In [6]:
# print(len(sample_data))
# print(sample_data[0])
# print(sample_data[1]['sig_len'])
# print(sample_data[1]['fs'])
# print(sample_data[1]['sig_name'])
sample_data

(array([[-0.145, -0.065],
        [-0.145, -0.065],
        [-0.145, -0.065],
        ...,
        [-0.675, -0.365],
        [-0.765, -0.335],
        [-1.28 ,  0.   ]]),
 {'fs': 360,
  'sig_len': 650000,
  'n_sig': 2,
  'base_date': None,
  'base_time': None,
  'units': ['mV', 'mV'],
  'sig_name': ['MLII', 'V5'],
  'comments': ['69 M 1085 1629 x1', 'Aldomet, Inderal']})

In [7]:
sample_fs = sample_data[1]['fs']
sample_leads = sample_data[1]['sig_name']

selected_lead_index = sample_leads.index('MLII')
print(selected_lead_index)
# selected_lead_index = sample_leads.index('V5')
# print(selected_lead_index)

sample_lead_data = sample_data[0][:, selected_lead_index]

print(len(sample_annotation['symbol']))
print(len(sample_annotation['sample']))
print(sample_annotation['sample'])
#print(sample_annotation['symbol'])

sample_index_list = list(sample_annotation['sample'])
sample_label_list = sample_annotation['symbol']


0
2274
2274
[    18     77    370 ... 649484 649734 649991]


In [8]:
sample_data = []
sample_labels = []

for i in range(len(sample_label_list)):
    sample_mark = sample_label_list[i]
    if sample_mark in label_group_map.keys():
        idx_start = sample_index_list[i] - int(sample_fs / 2)
        idx_end = sample_index_list[i] + int(sample_fs / 2)
        if 0 <= idx_start < len(sample_lead_data) and idx_end <= len(sample_lead_data):
            sample_segment = sample_lead_data[idx_start: idx_end]
            sample_segment_resampled = resample_interpolation(sample_segment, sample_fs, fs_out=360)
            sample_data.append(sample_segment_resampled)
            sample_labels.append(label_group_map[sample_mark])

print('data size: ', len(sample_data), '*', len(sample_data[0]))
print('label size: ', len(sample_labels))


data size:  2271 * 360
label size:  2271


In [9]:
from sklearn.preprocessing import LabelEncoder

sample_label_encoder = LabelEncoder()
sample_labels_encoded = sample_label_encoder.fit_transform(sample_labels)
print(set(sample_labels_encoded))


{0, 1, 2}


In [ ]:
#from sklearn.preprocessing import LabelEncoder

def load_and_preprocess_data(path, valid_lead=['MLII'], fs_out=360):
    all_data = []
    all_labels = []

    with open(os.path.join(path, 'RECORDS'), 'r') as fin:
        all_record_name = fin.read().strip().split('\n')

    for record_name in all_record_name:
        try:
            tmp_ann_res = wfdb.rdann(os.path.join(path, record_name), 'atr')
            tmp_data_res = wfdb.rdsamp(os.path.join(path, record_name))
        except:
            print(f'Error reading {record_name}')
            continue

        fs = tmp_data_res[1]['fs']
        lead_in_data = tmp_data_res[1]['sig_name']

        if valid_lead[0] in lead_in_data:
            channel = lead_in_data.index(valid_lead[0])
            tmp_data = tmp_data_res[0][:, channel]

            idx_list = list(tmp_ann_res.sample)
            label_list = tmp_ann_res.symbol

            for i in range(len(label_list)):
                s = label_list[i]
                if s in label_group_map.keys():
                    idx_start = idx_list[i] - int(fs / 2)
                    idx_end = idx_list[i] + int(fs / 2)
                    if 0 <= idx_start < len(tmp_data) and idx_end <= len(tmp_data):
                        segment = tmp_data[idx_start:idx_end]
                        segment_resampled = resample_interpolation(segment, fs, fs_out)
                        all_data.append(segment_resampled)
                        all_labels.append(label_group_map[s])

            print(f'Processed {record_name}, total segments: {len(all_data)}')

    label_encoder = LabelEncoder()
    all_labels_encoded = label_encoder.fit_transform(all_labels)

    return np.array(all_data), all_labels_encoded

path = r'D:\CodeProjects\VsCodePythonPro\MachinelearningClass\EX5\mit-bih-arrhythmia-database-1.0.0_data'
all_data, all_labels = load_and_preprocess_data(path)
print()
print(f'Preprocessed data shape: {all_data.shape}')
print(f'Preprocessed labels shape: {all_labels.shape}')

In [11]:
import numpy as np

unique_labels, counts = np.unique(all_labels, return_counts=True)

for label, count in zip(unique_labels, counts):
    print(f'Class {label}: {count} instances')


Class 0: 802 instances
Class 1: 90075 instances
Class 2: 3894 instances
Class 3: 3026 instances
Class 4: 7229 instances


## 3.feature Extraction

In [12]:
import numpy as np
from scipy.stats import skew, kurtosis

def extract_features(records):
    features = []
    for record in records:
        mean_val = np.mean(record)
        std_val = np.std(record)
        max_val = np.max(record)
        min_val = np.min(record)
        
        skewness = skew(record)
        kurt = kurtosis(record)
        
        sorted_neg_values = np.sort(record[record < 0])
        second_min_val = sorted_neg_values[1] if len(sorted_neg_values) > 1 else (sorted_neg_values[0] if len(sorted_neg_values) == 1 else np.nan)
        
        fft_vals = np.fft.fft(record)
        power_spectrum = np.mean(np.abs(fft_vals) ** 2)

        features.append([mean_val, std_val, max_val, min_val, skewness, kurt, second_min_val, power_spectrum])
    
    return np.array(features)

features = extract_features(all_data)

print(f'Extracted features shape: {features.shape}')
print(features[:5])


Extracted features shape: (105026, 8)
[[-0.30979167  0.16913103  0.94       -0.535       5.09997471 30.91398797
  -0.51       44.847425  ]
 [-0.33238889  0.15355008  0.96       -0.57        5.97456751 41.71478844
  -0.555      48.2616    ]
 [-0.33295833  0.14579737  0.86       -0.645       5.6542518  39.14871475
  -0.63       47.562525  ]
 [-0.33195833  0.13820973  0.82       -0.565       5.6519943  39.4829461
  -0.555      46.547375  ]
 [-0.32575     0.14692399  0.885      -0.545       5.52627975 37.60597187
  -0.54       45.9719    ]]


## 4.Split

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, all_labels, test_size=0.2, random_state=42)

print(f'Training data shape: {X_train.shape}, Training labels shape: {y_train.shape}')
print(f'Test data shape: {X_test.shape}, Test labels shape: {y_test.shape}')

Training data shape: (84020, 8), Training labels shape: (84020,)
Test data shape: (21006, 8), Test labels shape: (21006,)


In [14]:
np.save('X_train.npy', X_train)
np.save('X_test.npy', X_test)
np.save('y_train.npy', y_train)
np.save('y_test.npy', y_test)

In [15]:
X_train = np.load('X_train.npy')
X_test = np.load('X_test.npy')
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

## 5.Training

In [16]:
from sklearn.ensemble import VotingClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

def train_ensemble_voting(X_train, y_train):
    imputer = SimpleImputer(strategy='mean')
    
    svm_model = Pipeline([
        ('imputer', imputer),
        ('classifier', SVC(probability=True, random_state=44, class_weight='balanced'))
    ])
    
    rf_model = Pipeline([
        ('imputer', imputer),
        ('classifier', RandomForestClassifier(random_state=44, class_weight='balanced'))
    ])
    
    ensemble_model = VotingClassifier(estimators=[
        ('svm', svm_model),
        ('rf', rf_model)], voting='soft')

    ensemble_model.fit(X_train, y_train)
    return ensemble_model

ensemble_model = train_ensemble_voting(X_train, y_train)

print('Ensemble model trained successfully')


Ensemble model trained successfully


## 6.Evaluation

In [17]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred = ensemble_model.predict(X_test)

def evaluate_model(y_test, y_pred):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='macro')
    recall = recall_score(y_test, y_pred, average='macro')
    f1 = f1_score(y_test, y_pred, average='macro')
    
    print(f'Accuracy: {accuracy:.2f}')
    print(f'Precision: {precision:.2f}')
    print(f'Recall: {recall:.2f}')
    print(f'F1 Score: {f1:.2f}')

evaluate_model(y_test, y_pred)


Accuracy: 0.94
Precision: 0.92
Recall: 0.59
F1 Score: 0.66
